In [1]:
import serial
from threading import Thread
import struct
import numpy as np
import pinocchio as pin
import meshcat
from scipy.spatial.transform import Rotation
import time

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
class ExoIMUs:
    def __init__(self, port = '/dev/ttyACM2'):
        self.port = port
        self.serial = serial.Serial(port, baudrate=115200, timeout=1)
        self.running = True
        self.state = None
        self.thread = Thread(target=self.update)
        self.thread.start()

    def update(self):
        while self.running:
            data = self.serial.read_until(b'abc\n')
            if(len(data) == 4 + 3*(4*4 + 4)):
                data = struct.unpack('15f',data[:-4])
                self.state = {'q_hand':np.asarray(data[0:4]),
                            'q_shoulder':np.asarray(data[4:8]),
                            'q_base':np.asarray(data[8:12]),
                            'hand_acc': data[12],
                            'shoulder_acc': data[13],
                            'base_acc':data[14]}
            
    def read(self):
        return self.state
    
    def close(self):
        self.running=False
        self.serial.close()

imu = ExoIMUs()

SerialException: [Errno 2] could not open port /dev/ttyACM0: [Errno 2] No such file or directory: '/dev/ttyACM0'

In [8]:
imu.read()

{'q_hand': array([0.05194092, 0.07574463, 0.60681152, 0.78955078]),
 'q_shoulder': array([ 0.07855225,  0.35845947, -0.52252197,  0.76965332]),
 'q_base': array([0.49591064, 0.01171875, 0.07226562, 0.86529541]),
 'hand_acc': 0.0,
 'shoulder_acc': 3.0,
 'base_acc': 3.0}

In [ ]:

def add_frame(name, vis):
    xbox = meshcat.geometry.Box([0.1, 0.01, 0.01])
    vis["xbox_" + name].set_object(xbox, meshcat.geometry.MeshLambertMaterial(
                                    color=0xFF0000))
    ybox = meshcat.geometry.Box([0.01, 0.1, 0.01])
    vis["ybox_" + name].set_object(ybox, meshcat.geometry.MeshLambertMaterial(
                                    color=0x00FF00))
    zbox = meshcat.geometry.Box([0.01, 0.01, 0.1])
    vis["zbox_" + name].set_object(zbox, meshcat.geometry.MeshLambertMaterial(
                                    color=0x0000FF))

def update_frame(name, vis, R, offset = np.zeros(3)):
    X_TG = np.eye(4)
    X_TG[0,3] = 0.05
    Y_TG = np.eye(4)
    Y_TG[1,3] = 0.05
    Z_TG = np.eye(4)
    Z_TG[2,3] = 0.05

    offset_TG = np.eye(4)
    offset_TG[0:3,3] = offset


    T = np.eye(4)
    T[0:3,0:3] = R
    vis["xbox_" + name].set_transform( offset_TG @ T @ X_TG )
    vis["ybox_" + name].set_transform( offset_TG @ T @ Y_TG )
    vis["zbox_" + name].set_transform( offset_TG @ T @ Z_TG )


NameError: name 'np' is not defined

In [ ]:
vis = meshcat.Visualizer()

add_frame("hand", vis)
add_frame("base", vis)
add_frame("shoulder", vis)
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [ ]:
names = ['hand', 'base', 'shoulder']
while True:
    data = imu.read()
    for name in names:
        iRbr1 = Rotation.from_quat(data['q_' + name]).as_matrix()
        update_frame(name, vis, iRbr1)
    time.sleep(0.01)

KeyboardInterrupt: 